In [1]:
# importing libraries for data modeling

import pandas as pd
import numpy as np
import collections
from nltk.tokenize.treebank import TreebankWordTokenizer
from nltk.tokenize import TweetTokenizer
from common import utils, vocabulary
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import preprocessing
import tensorflow as tf
import os
import time
import datetime
from CNN.text_cnn import TextCNN
from tensorflow.contrib import learn
import sys

/home/chadharness/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [1]:
!pwd

/home/chadharness/w210/Parlancr/models/classifier


In [3]:
# loading model data 

mod_train = pd.read_csv("../../../language-style-transfer-test-v2/language-style-transfer/data/yelp/shakes_mod.train.0", delimiter="\t", header = None, names = ['sentence_text'])
shakes_train = pd.read_csv("../../../language-style-transfer-test-v2/language-style-transfer/data/yelp/shakes_mod.train.1", delimiter="\t", header = None, names = ['sentence_text'])
mod_test = pd.read_csv("../../../language-style-transfer-test-v2/language-style-transfer/data/yelp/shakes_mod.test.0", delimiter="\t", header = None, names = ['sentence_text'])
shakes_test = pd.read_csv("../../../language-style-transfer-test-v2/language-style-transfer/data/yelp/shakes_mod.test.1", delimiter="\t", header = None, names = ['sentence_text'])

mod_train['group']='modern'
mod_test['group']='modern'
shakes_train['group']='shakespeare'
shakes_test['group']='shakespeare'

#combine dem/rep data together for training/testing
#shuffle it for random order
all_train = mod_train.append(shakes_train)
all_test = mod_test.append(shakes_test)

all_train=all_train.sample(frac=1)
all_test=all_test.sample(frac=1)

print(all_train.head())
print(all_test.head())

                                           sentence_text        group
16795          why , sir , what's your conceit in that ?  shakespeare
14394           why do you keep repeating that , woman ?       modern
1072                                        is he dead ?       modern
8481   twice then the trumpets sounded , and there i ...  shakespeare
13651  take it and do ' t , and leave me for this time .  shakespeare
                                          sentence_text        group
556   some say the lark makes a sweet division betwe...       modern
141   now is the sun upon the highmost hill of this ...  shakespeare
1431                          good night , good night .       modern
787                                      i'm not here .       modern
105   believe me , i'll draw my sword as quick as an...       modern


In [4]:
print(all_train.shape)
print(all_test.shape)

(36790, 2)
(2924, 2)


In [5]:
smpl_train = all_train.sample(frac=1)
smpl_test = all_test.sample(frac=1)
print(smpl_train.shape)
print(smpl_test.shape)

(36790, 2)
(2924, 2)


In [6]:
train_x = smpl_train['sentence_text']
train_y = smpl_train['group']
test_x = smpl_test['sentence_text']
test_y = smpl_test['group']

In [7]:
# tokenize data in comment body

tokenizer = TweetTokenizer()

train_x_tokens = []
test_x_tokens = []

for x in train_x:
    train_x_tokens.append(tokenizer.tokenize(x))
    
for x in test_x:
    test_x_tokens.append(tokenizer.tokenize(x))

In [8]:
# define vocabulary using w266 common vocab

train_text_all = [item for sublist in train_x_tokens for item in sublist]

vocab = vocabulary.Vocabulary(train_text_all, size=20000)
print("Vocabulary size: {:,}".format(vocab.size))
print("Vocabulary dict: ", vocab.word_to_id)

train_x_ids = []
test_x_ids = []

for x in train_x_tokens:
    train_x_ids.append(vocab.words_to_ids(x))
    
for x in test_x_tokens:
    test_x_ids.append(vocab.words_to_ids(x))

Vocabulary size: 15,357
Vocabulary dict:  {'<s>': 0, '</s>': 1, '<unk>': 2, ',': 3, '.': 4, 'the': 5, 'i': 6, 'and': 7, 'you': 8, 'to': 9, '?': 10, 'a': 11, 'my': 12, 'of': 13, 'that': 14, 'me': 15, 'is': 16, 'in': 17, 'it': 18, '!': 19, 'your': 20, 'for': 21, 'not': 22, 'be': 23, 'he': 24, 'this': 25, 'with': 26, 'have': 27, 'but': 28, 'as': 29, 'what': 30, 'his': 31, 'him': 32, 'will': 33, 'so': 34, 'do': 35, 'if': 36, 'are': 37, 'her': 38, 'on': 39, 'no': 40, 'all': 41, 'good': 42, 'we': 43, 'lord': 44, 'thou': 45, 'now': 46, "'": 47, "i'll": 48, 'by': 49, 'here': 50, 'come': 51, 'go': 52, 'was': 53, '"': 54, 'our': 55, 'at': 56, 'like': 57, 'sir': 58, 'well': 59, 'can': 60, 'from': 61, 'then': 62, 'they': 63, 'she': 64, 'how': 65, 'when': 66, 'more': 67, 'would': 68, 'or': 69, 'know': 70, 'man': 71, 'who': 72, 'let': 73, 'them': 74, 'there': 75, 'shall': 76, 'love': 77, 'say': 78, 'tell': 79, 'am': 80, 'out': 81, 'us': 82, "i'm": 83, 'oh': 84, 'did': 85, 'than': 86, 'one': 87, 'why

In [9]:
# count token occurences and convert to feature vector for BoW model

train_x_fdict = []
test_x_fdict = []

for x in train_x_ids:
    train_x_fdict.append(collections.Counter(x))
    
for x in test_x_ids:
    test_x_fdict.append(collections.Counter(x))

train_x_vector = []
test_x_vector = []

num_features = vocab.size
print('creating id lists...')
for x in train_x_fdict:
    train_x_vector.append([x.get(i, 0) for i in range(num_features)])
    
for x in test_x_fdict:
    test_x_vector.append([x.get(i, 0) for i in range(num_features)])

# use w266 common utils to convert id lists to sparse bow matrix
print('starting id lists to sparse bow conversion...')
train_x_sparse_bow = utils.id_lists_to_sparse_bow(train_x_fdict, vocab.size)
test_x_sparse_bow = utils.id_lists_to_sparse_bow(test_x_fdict, vocab.size)

# training Multinomial Naive Bayes for simple baseline model
print('training Multinomial Naive Bayes for simple baseline model...')
nb = MultinomialNB()
nb.fit(train_x_sparse_bow, train_y)
y_pred = nb.predict(test_x_sparse_bow)

acc = accuracy_score(test_y, y_pred)
print("Accuracy on test set: {:.02%}".format(acc))
classrep = classification_report(test_y, y_pred)
print(classrep)

creating id lists...
starting id lists to sparse bow conversion...
training Multinomial Naive Bayes for simple baseline model...
Accuracy on test set: 82.52%
             precision    recall  f1-score   support

     modern       0.83      0.82      0.83      1462
shakespeare       0.82      0.83      0.83      1462

avg / total       0.83      0.83      0.83      2924



In [10]:
# pad all id vectors to max length for Neural model

max_comment_length = max([len(x) for x in train_x_ids])

train_x_neural = train_x_ids
test_x_neural = test_x_ids

for x in train_x_neural:
    if len(x) < max_comment_length:
        for n in range(max_comment_length - len(x)):
            x.append(1)
    elif len(x) > max_comment_length:
        x = x[:max_comment_length]
        
for x in test_x_neural:
    if len(x) < max_comment_length:
        for n in range(max_comment_length - len(x)):
            x.append(1)
    elif len(x) > max_comment_length:
        x = x[:max_comment_length]

# convert labels to one-hot vectors for Neural model --update your class lables below!!

terms_list = [['modern'],['shakespeare']]
lb = preprocessing.MultiLabelBinarizer()
lb.fit(terms_list)

train_labels = np.array(train_y.tolist())
test_labels = np.array(test_y.tolist())

train_lables_format = [train_labels[i:i+1] for i in range(0, len(train_labels), 1)]    
test_lables_format = [test_labels[i:i+1] for i in range(0, len(test_labels), 1)]

train_y_neural = lb.transform(train_lables_format)
test_y_neural = lb.transform(test_lables_format)

##### The best run:
2018-08-05T21:53:47.670865: step 3500, loss 0.355526, acc 0.826265

Saved model checkpoint to /home/chadharness/w210/Parlancr/models/classifier/runs/1533505405/checkpoints/model-3500

In [11]:
# creating and training CNN for deep classification
# CNN model structure and code based on an implementation of Kim Yoon's "Convolutional Neural Networks for Sentence Classification"
# From https://github.com/dennybritz/cnn-text-classification-tf

# Parameters
# ==================================================

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 20, "Number of checkpoints to store (default: 5)")

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
tf.app.flags.DEFINE_string('f', '', 'kernel')

FLAGS = tf.flags.FLAGS
FLAGS(sys.argv)
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


# Data Preparation
# ==================================================

x_train = np.array(train_x_neural)
y_train = np.array(train_y_neural)
x_dev = np.array(test_x_neural)
y_dev = np.array(test_y_neural)

print("x_train", np.shape(x_train))
print("y_train", np.shape(y_train))
print("x_dev", np.shape(x_dev))
print("x_dev", np.shape(y_dev))

print("Vocabulary Size: {:d}".format(vocab.size))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))


# Training
# ==================================================

def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]
            
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=2,
            vocab_size=vocab.size,
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))


Parameters:
ALLOW_SOFT_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x7f67d36cc2e8>
BATCH_SIZE=<absl.flags._flag.Flag object at 0x7f67d36c4eb8>
CHECKPOINT_EVERY=<absl.flags._flag.Flag object at 0x7f67d36cc0b8>
DROPOUT_KEEP_PROB=<absl.flags._flag.Flag object at 0x7f67d36c4e48>
EMBEDDING_DIM=<absl.flags._flag.Flag object at 0x7f67d36c4c50>
EVALUATE_EVERY=<absl.flags._flag.Flag object at 0x7f67d36cc0f0>
F=<absl.flags._flag.Flag object at 0x7f67d36cc438>
FILTER_SIZES=<absl.flags._flag.Flag object at 0x7f67d36c4b38>
L2_REG_LAMBDA=<absl.flags._flag.Flag object at 0x7f67d36c4da0>
LOG_DEVICE_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x7f67d36cc390>
NUM_CHECKPOINTS=<absl.flags._flag.Flag object at 0x7f67d36cc160>
NUM_EPOCHS=<absl.flags._flag.Flag object at 0x7f67d36cc048>
NUM_FILTERS=<absl.flags._flag.Flag object at 0x7f67d36c4c88>

x_train (36790, 156)
y_train (36790, 2)
x_dev (2924, 156)
x_dev (2924, 2)
Vocabulary Size: 15357
Train/Dev split: 36790/2924
Instructions for updating

2018-08-05T21:43:38.438935: step 77, loss 1.03053, acc 0.640625
2018-08-05T21:43:38.596623: step 78, loss 1.13653, acc 0.578125
2018-08-05T21:43:38.755671: step 79, loss 1.51591, acc 0.515625
2018-08-05T21:43:38.915877: step 80, loss 1.39901, acc 0.59375
2018-08-05T21:43:39.078150: step 81, loss 1.23438, acc 0.53125
2018-08-05T21:43:39.240688: step 82, loss 1.33597, acc 0.53125
2018-08-05T21:43:39.409475: step 83, loss 1.36715, acc 0.5
2018-08-05T21:43:39.569085: step 84, loss 1.28722, acc 0.515625
2018-08-05T21:43:39.732076: step 85, loss 1.05902, acc 0.65625
2018-08-05T21:43:39.891279: step 86, loss 1.33842, acc 0.546875
2018-08-05T21:43:40.052224: step 87, loss 1.29639, acc 0.578125
2018-08-05T21:43:40.209655: step 88, loss 1.14397, acc 0.578125
2018-08-05T21:43:40.372297: step 89, loss 1.16312, acc 0.53125
2018-08-05T21:43:40.534563: step 90, loss 1.03024, acc 0.65625
2018-08-05T21:43:40.694669: step 91, loss 1.31351, acc 0.546875
2018-08-05T21:43:40.853929: step 92, loss 1.29733, 

2018-08-05T21:44:01.873195: step 201, loss 0.681501, acc 0.65625
2018-08-05T21:44:02.029140: step 202, loss 0.92697, acc 0.546875
2018-08-05T21:44:02.190435: step 203, loss 0.801912, acc 0.703125
2018-08-05T21:44:02.351419: step 204, loss 0.8999, acc 0.578125
2018-08-05T21:44:02.510774: step 205, loss 0.686235, acc 0.671875
2018-08-05T21:44:02.670007: step 206, loss 0.793747, acc 0.640625
2018-08-05T21:44:02.834206: step 207, loss 0.948668, acc 0.59375
2018-08-05T21:44:02.991624: step 208, loss 1.09906, acc 0.5
2018-08-05T21:44:03.154416: step 209, loss 0.971871, acc 0.578125
2018-08-05T21:44:03.315519: step 210, loss 1.04021, acc 0.53125
2018-08-05T21:44:03.480436: step 211, loss 0.964676, acc 0.578125
2018-08-05T21:44:03.638396: step 212, loss 0.997038, acc 0.5625
2018-08-05T21:44:03.800172: step 213, loss 0.800241, acc 0.59375
2018-08-05T21:44:03.967527: step 214, loss 0.859812, acc 0.625
2018-08-05T21:44:04.130492: step 215, loss 0.800289, acc 0.5625
2018-08-05T21:44:04.293865: ste

2018-08-05T21:44:23.521739: step 325, loss 0.509996, acc 0.765625
2018-08-05T21:44:23.683655: step 326, loss 0.895832, acc 0.578125
2018-08-05T21:44:23.843555: step 327, loss 0.709531, acc 0.578125
2018-08-05T21:44:24.003159: step 328, loss 0.87642, acc 0.65625
2018-08-05T21:44:24.166106: step 329, loss 0.764927, acc 0.578125
2018-08-05T21:44:24.329075: step 330, loss 0.927471, acc 0.53125
2018-08-05T21:44:24.493078: step 331, loss 0.775363, acc 0.65625
2018-08-05T21:44:24.654162: step 332, loss 0.798821, acc 0.609375
2018-08-05T21:44:24.817647: step 333, loss 0.727008, acc 0.609375
2018-08-05T21:44:24.975418: step 334, loss 0.699528, acc 0.640625
2018-08-05T21:44:25.140777: step 335, loss 0.596754, acc 0.71875
2018-08-05T21:44:25.304553: step 336, loss 0.67791, acc 0.625
2018-08-05T21:44:25.464407: step 337, loss 0.697874, acc 0.625
2018-08-05T21:44:25.622107: step 338, loss 0.652201, acc 0.609375
2018-08-05T21:44:25.778603: step 339, loss 0.808196, acc 0.625
2018-08-05T21:44:25.94280

2018-08-05T21:44:45.087636: step 449, loss 0.52582, acc 0.734375
2018-08-05T21:44:45.248789: step 450, loss 0.668648, acc 0.734375
2018-08-05T21:44:45.410791: step 451, loss 0.639823, acc 0.609375
2018-08-05T21:44:45.569353: step 452, loss 0.714545, acc 0.59375
2018-08-05T21:44:45.731669: step 453, loss 0.642911, acc 0.65625
2018-08-05T21:44:45.890380: step 454, loss 0.62086, acc 0.75
2018-08-05T21:44:46.048381: step 455, loss 0.776827, acc 0.515625
2018-08-05T21:44:46.208142: step 456, loss 0.618436, acc 0.703125
2018-08-05T21:44:46.369136: step 457, loss 0.616779, acc 0.6875
2018-08-05T21:44:46.529646: step 458, loss 0.846627, acc 0.59375
2018-08-05T21:44:46.690201: step 459, loss 0.733335, acc 0.609375
2018-08-05T21:44:46.849637: step 460, loss 0.701964, acc 0.515625
2018-08-05T21:44:47.007401: step 461, loss 0.746633, acc 0.59375
2018-08-05T21:44:47.167790: step 462, loss 0.524357, acc 0.71875
2018-08-05T21:44:47.328263: step 463, loss 0.536507, acc 0.703125
2018-08-05T21:44:47.485

...

2018-08-05T21:58:04.195518: step 4953, loss 0.22833, acc 0.90625
2018-08-05T21:58:04.360114: step 4954, loss 0.291801, acc 0.828125
2018-08-05T21:58:04.518559: step 4955, loss 0.144583, acc 0.9375
2018-08-05T21:58:04.684293: step 4956, loss 0.108218, acc 0.984375
2018-08-05T21:58:04.854522: step 4957, loss 0.196023, acc 0.921875
2018-08-05T21:58:05.018853: step 4958, loss 0.211327, acc 0.90625
2018-08-05T21:58:05.180506: step 4959, loss 0.252034, acc 0.90625
2018-08-05T21:58:05.344224: step 4960, loss 0.217656, acc 0.90625
2018-08-05T21:58:05.506164: step 4961, loss 0.230593, acc 0.90625
2018-08-05T21:58:05.666606: step 4962, loss 0.183585, acc 0.9375
2018-08-05T21:58:05.830035: step 4963, loss 0.188098, acc 0.921875
2018-08-05T21:58:05.991811: step 4964, loss 0.245962, acc 0.921875
2018-08-05T21:58:06.149944: step 4965, loss 0.219946, acc 0.9375
2018-08-05T21:58:06.308561: step 4966, loss 0.173566, acc 0.90625
2018-08-05T21:58:06.469855: step 4967, loss 0.117513, acc 0.96875
2018-08-0

2018-08-05T21:58:25.440853: step 5075, loss 0.135572, acc 0.953125
2018-08-05T21:58:25.603180: step 5076, loss 0.161766, acc 0.90625
2018-08-05T21:58:25.761118: step 5077, loss 0.236082, acc 0.890625
2018-08-05T21:58:25.929380: step 5078, loss 0.109181, acc 0.953125
2018-08-05T21:58:26.092610: step 5079, loss 0.267225, acc 0.890625
2018-08-05T21:58:26.249497: step 5080, loss 0.30831, acc 0.890625
2018-08-05T21:58:26.409582: step 5081, loss 0.348077, acc 0.78125
2018-08-05T21:58:26.572672: step 5082, loss 0.156775, acc 0.96875
2018-08-05T21:58:26.736848: step 5083, loss 0.139068, acc 0.953125
2018-08-05T21:58:26.896259: step 5084, loss 0.214963, acc 0.875
2018-08-05T21:58:27.056277: step 5085, loss 0.181392, acc 0.90625
2018-08-05T21:58:27.213482: step 5086, loss 0.209495, acc 0.9375
2018-08-05T21:58:27.373273: step 5087, loss 0.205192, acc 0.90625
2018-08-05T21:58:27.530980: step 5088, loss 0.13615, acc 0.96875
2018-08-05T21:58:27.693353: step 5089, loss 0.301029, acc 0.859375
2018-08-

2018-08-05T21:58:46.732001: step 5197, loss 0.173939, acc 0.90625
2018-08-05T21:58:46.890688: step 5198, loss 0.197802, acc 0.90625
2018-08-05T21:58:47.053030: step 5199, loss 0.196243, acc 0.875
2018-08-05T21:58:47.218031: step 5200, loss 0.156971, acc 0.9375

Evaluation:
2018-08-05T21:58:48.690548: step 5200, loss 0.393816, acc 0.832421

Saved model checkpoint to /home/chadharness/w210/Parlancr/models/classifier/runs/1533505405/checkpoints/model-5200

2018-08-05T21:58:48.921559: step 5201, loss 0.195217, acc 0.90625
2018-08-05T21:58:49.074748: step 5202, loss 0.251138, acc 0.875
2018-08-05T21:58:49.230532: step 5203, loss 0.115985, acc 0.984375
2018-08-05T21:58:49.390795: step 5204, loss 0.251539, acc 0.84375
2018-08-05T21:58:49.553748: step 5205, loss 0.184776, acc 0.921875
2018-08-05T21:58:49.716531: step 5206, loss 0.107237, acc 0.953125
2018-08-05T21:58:49.881560: step 5207, loss 0.127549, acc 0.96875
2018-08-05T21:58:50.046549: step 5208, loss 0.0922444, acc 0.953125
2018-08-05T

2018-08-05T21:59:09.159000: step 5317, loss 0.153137, acc 0.9375
2018-08-05T21:59:09.315992: step 5318, loss 0.185358, acc 0.90625
2018-08-05T21:59:09.477668: step 5319, loss 0.144078, acc 0.921875
2018-08-05T21:59:09.639910: step 5320, loss 0.191958, acc 0.90625
2018-08-05T21:59:09.806216: step 5321, loss 0.120324, acc 0.96875
2018-08-05T21:59:09.966845: step 5322, loss 0.115635, acc 0.953125
2018-08-05T21:59:10.131934: step 5323, loss 0.119156, acc 0.9375
2018-08-05T21:59:10.286422: step 5324, loss 0.20234, acc 0.875
2018-08-05T21:59:10.447722: step 5325, loss 0.251191, acc 0.90625
2018-08-05T21:59:10.608783: step 5326, loss 0.163228, acc 0.921875
2018-08-05T21:59:10.768174: step 5327, loss 0.176115, acc 0.9375
2018-08-05T21:59:10.928349: step 5328, loss 0.088825, acc 0.984375
2018-08-05T21:59:11.090439: step 5329, loss 0.109627, acc 0.96875
2018-08-05T21:59:11.251992: step 5330, loss 0.190176, acc 0.921875
2018-08-05T21:59:11.410148: step 5331, loss 0.171558, acc 0.921875
2018-08-05

KeyboardInterrupt: 

In [ ]:
# run this to clear tf flags without re-starting kernel

def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)
        
del_all_flags(tf.flags.FLAGS)

## Testing the Shen model results

In [25]:
# loading model data 
mod_shakes_style_tsf = pd.read_csv("../../results/shakespeare/shakes_mod.test.0.tsf", delimiter="\n", quoting=3, header = None, names = ['sentence_text'])
shakes_mod_style_tsf = pd.read_csv("../../results/shakespeare/shakes_mod.test.1.tsf", delimiter="\n", quoting=3, header = None, names = ['sentence_text'])

mod_shakes_style_tsf['group']='shakespeare'
shakes_mod_style_tsf['group']='modern'

#combine dem/rep data together for training/testing
#shuffle it for random order
all_transf = mod_shakes_style_tsf.append(shakes_mod_style_tsf)

all_transf=all_transf.sample(frac=1)

print(all_transf.head())

                                          sentence_text        group
719   come to me , my lord . , let me alone . , come...       modern
1029               come on me , and the <unk> of this !  shakespeare
553                                that's not neither .  shakespeare
322                where are you in the <unk> of this ?       modern
1311         you know me with her , and let her <unk> .  shakespeare


In [26]:
print(all_transf.shape)

(2924, 2)


In [27]:
test_x = all_transf['sentence_text']
test_y = all_transf['group']

In [28]:
# tokenize data in comment body

tokenizer = TweetTokenizer()

test_x_tokens = []
    
for x in test_x:
    test_x_tokens.append(tokenizer.tokenize(x))

In [30]:
test_x_ids = []

for x in test_x_tokens:
    test_x_ids.append(vocab.words_to_ids(x))

test_x_fdict = []
    
for x in test_x_ids:
    test_x_fdict.append(collections.Counter(x))

test_x_vector = []

num_features = vocab.size
print('creating id lists...')

for x in test_x_fdict:
    test_x_vector.append([x.get(i, 0) for i in range(num_features)])

# use w266 common utils to convert id lists to sparse bow matrix
print('starting id lists to sparse bow conversion...')
test_x_sparse_bow = utils.id_lists_to_sparse_bow(test_x_fdict, vocab.size)

# use Multinomial Naive Bayes to classify transfer sentences
print('generating predictions on transferred data...')
y_pred = nb.predict(test_x_sparse_bow)

acc = accuracy_score(test_y, y_pred)
print("Accuracy on test set: {:.02%}".format(acc))
classrep = classification_report(test_y, y_pred)
print(classrep)

creating id lists...
starting id lists to sparse bow conversion...
generating predictions on transferred data...
Accuracy on test set: 67.03%
             precision    recall  f1-score   support

     modern       0.65      0.73      0.69      1462
shakespeare       0.69      0.61      0.65      1462

avg / total       0.67      0.67      0.67      2924

